In [2]:
import sys
sys.path.append('../')
import lib
import spacy
import json
import pandas as pd
import os
os.listdir('../data')

['tweets_Gabonews_com_2020-02-01_2020-04-14.json',
 '.articles_CorbeauNews_2020-02-01_2020-04-14.json.icloud',
 '.articles_QuotidienSN_2020-01-01_2020-04-14.json.icloud',
 'lequotidien.json',
 'articles_SAVOIRNEWS1_2020-02-01_2020-04-14.json',
 'tweets_Lefaso_net_2020-02-01_2020-04-14.json',
 '.articles_CongoActu_2020-02-01_2020-04-14.json.icloud',
 'lesoleil.json',
 'tweets_JourDuMali_2020-02-01_2020-04-14.json',
 'tweets_LeSoleilonline_2020-02-02_2020-04-12.json',
 'tweets_CongoActu_2020-02-01_2020-04-14.json',
 'articles_iwacuinfo_2020-02-01_2020-04-14.json',
 'articles_JourDuMali_2020-02-01_2020-04-14.json',
 '.tweets_iwacuinfo_2020-02-01_2020-04-14.json.icloud',
 'articles_Lefaso_net_2020-02-01_2020-04-14.json',
 'articles_comoresinfos_2020-02-01_2020-04-14.json',
 '.tweets_comoresinfos_2020-02-01_2020-04-14.json.icloud',
 'articles_madatribune_2020-02-01_2020-04-14.json',
 'Macky_Sall_twitter.json',
 'tweets_InfosGabon_2020-02-01_2020-04-14.json',
 'articles_ICIBrazza_2020-02-01_

In [3]:
#example_path = '../data/articles_ICIBrazza_2020-02-01_2020-04-14.json'
example_article = 'articles_SAVOIRNEWS1_2020-02-01_2020-04-14.json'
example_path = os.path.join('../data', example_article)

with open(example_path, 'r') as reader:
    data = json.load(reader)
df = pd.DataFrame(columns=data['head'], 
                 data=data['articles'])
print(len(df))
df.head()

46


,id,title,text,Date,country,source,tweet
0,Togo_SAVOIRNEWS1_0,Togo/Coronavirus : Six nouveaux cas ce lundi d...,Six nouveaux cas confirmés au coronavirus dont...,2020-04-06T23:34:57,Togo,SAVOIRNEWS1,*Six nouveaux cas confirmés au coronavirus don...
1,Togo_SAVOIRNEWS1_1,Togo/Coronavirus : Six nouveaux cas ce lundi d...,Six nouveaux cas confirmés au coronavirus dont...,2020-04-06T23:34:57,Togo,SAVOIRNEWS1,*Six nouveaux cas confirmés au coronavirus don...
2,Togo_SAVOIRNEWS1_2,Coronavirus: Les Évêques du Togo dictent des d...,"Les Évêques du Togo ont annoncé lundi, une sér...",2020-03-17T16:53:44,Togo,SAVOIRNEWS1,Pandémie de coronavirus :\nLes Évêques du #Tog...
3,Togo_SAVOIRNEWS1_3,LA PREMIERE AGENCE DE PRESSE PRIVEE AU TOGO,Faure Gnassingbé : « La victoire dans la lutte...,2020-03-15T13:51:10,Togo,SAVOIRNEWS1,Le nouveau coronavirus a fait au moins 5.402 m...
4,Togo_SAVOIRNEWS1_4,Coronavirus: Les Évêques du Togo dictent des d...,"Les Évêques du Togo ont annoncé lundi, une sér...",2020-03-17T16:53:44,Togo,SAVOIRNEWS1,Pandémie de coronavirus :\nLes Évêques du #Tog...


# Clean up the text for the EDA
    - Remove duplicate and potential NaNs
    - Remove index
    - Change types 
    - Remove part of the punctuation, and unicode typos

In [4]:
#Basic cleaning of the dataframe
clean_df = df.drop('id', axis=1).drop_duplicates(keep='first')
clean_df = clean_df[~clean_df['Date'].isna()]
clean_df

,title,text,Date,country,source,tweet
0,Togo/Coronavirus : Six nouveaux cas ce lundi d...,Six nouveaux cas confirmés au coronavirus dont...,2020-04-06T23:34:57,Togo,SAVOIRNEWS1,*Six nouveaux cas confirmés au coronavirus don...
2,Coronavirus: Les Évêques du Togo dictent des d...,"Les Évêques du Togo ont annoncé lundi, une sér...",2020-03-17T16:53:44,Togo,SAVOIRNEWS1,Pandémie de coronavirus :\nLes Évêques du #Tog...
3,LA PREMIERE AGENCE DE PRESSE PRIVEE AU TOGO,Faure Gnassingbé : « La victoire dans la lutte...,2020-03-15T13:51:10,Togo,SAVOIRNEWS1,Le nouveau coronavirus a fait au moins 5.402 m...
6,Changements climatiques en Afrique : Réunies à...,Plus de 300 journalistes africaines « Les Pana...,2020-03-07T07:30:42,Togo,SAVOIRNEWS1,Urgence climatique : Les journalistes panafric...
8,Social/Pour la continuité du travail : Pas de ...,Gilbert Bawara (ministre de la fonction publiq...,2020-03-31T15:52:36,Togo,SAVOIRNEWS1,Gilbert Bawara (ministre de la fonction publiq...
10,"Coronavirus : Des mesures pour des ""réponses e...",Le Togo a déjà enregistré 30 cas dont un décès...,2020-03-29T22:05:35,Togo,SAVOIRNEWS1,Le gouvernement entend apporter de nouvelles m...
11,"Coronavirus: Le Bénin n'a pas les ""moyens"" de ...",Le Bénin ne dispose pas des « moyens des pays ...,2020-03-29T22:02:30,Togo,SAVOIRNEWS1,Le Bénin ne dispose pas des « moyens des pays ...
12,Coronavirus: Le premier décès au Togo (officiel),"Le Togo a enregistré vendredi, son premier déc...",2020-03-28T13:00:25,Togo,SAVOIRNEWS1,#Covid19TGhttp://www.savoirnews.net/coronaviru...
16,Covid-19/Crise économique : Les médias privés ...,"Activités totalement au ralenti, absence d’ann...",2020-04-04T19:13:29,Togo,SAVOIRNEWS1,"Activités totalement au ralenti, absence d’ann..."
17,"Togo/Coronavirus: 40 cas avec 3 morts, Lomé s’...","Le Togo a enregistré ce vendredi, son 3ème déc...",2020-04-03T18:25:21,Togo,SAVOIRNEWS1,"Le Togo a enregistré ce vendredi, son 3ème déc..."


In [5]:
#id_ = ['Congo_ICIBrazza_' + str(i) for i in range(len(clean_df))]
#index = list(range(len(clean_df)))
def clean_index_text(df):
    """
    Clean up the id part and replace empty 
    articles by tweets.
    """
    id_name = "{}_{}_".format(df.iloc[0].country, df.iloc[0].source)
    id_ = [id_name + str(i) for i in range(len(df))]
    df['id'] = id_
    for i in range(len(df)):
        if df.iloc[i].text == '':
            print(f'Empty text for {i}, use tweet instead')
            df.loc[i,'text'] = df.iloc[i]['tweet']
    return df

clean_df = clean_index_text(clean_df)
#clean_df = clean_df[~clean_df['Date'].isna()]
#clean_df.iloc[8].text.replace('#', '')
#clean_df.iloc[7]


Empty text for 14, use tweet instead


In [6]:
#Original text (We have some \n, and potential other codes)
clean_df = clean_df[~clean_df['Date'].isna()]
clean_df.tail()

,title,text,Date,country,source,tweet,id
36,Présidentielle au Togo: Faure Gnassingbé réélu...,Faure Essozimna Gnassingbé a été réélu avec 72...,2020-02-24T02:52:34,Togo,SAVOIRNEWS1,Faure Essozimna Gnassingbé a été réélu avec 72...,Togo_SAVOIRNEWS1_18
37,Présidentielle : Pouvoir et opposition revendi...,Le parti au pouvoir et le candidat Agbéyomé Ko...,2020-02-23T20:53:05,Togo,SAVOIRNEWS1,Le parti au pouvoir et le candidat Agbéyomé Ko...,Togo_SAVOIRNEWS1_19
38,"Présidentielle au Togo: ""Une avance irrattrapa...",Le président Faure Gnassingbé a enregistré une...,2020-02-23T20:49:31,Togo,SAVOIRNEWS1,http://www.savoirnews.net/presidentielle-au-to...,Togo_SAVOIRNEWS1_20
39,Présidentielle : Le gouvernement dénonce le co...,"Le gouvernement a dénoncé ce samedi, le compor...",2020-02-23T19:53:43,Togo,SAVOIRNEWS1,"Le gouvernement a dénoncé ce samedi, le compor...",Togo_SAVOIRNEWS1_21
40,Présidentielle au Togo: Le président de la Cén...,"Tchambakou Ayassor, président de la Commission...",2020-02-22T20:42:53,Togo,SAVOIRNEWS1,"Plus de 3,6 millions de Togolais étaient appel...",Togo_SAVOIRNEWS1_22


# Text and Topic Analysis
    - Cleaning up our data (frames) is now done and we can start analyzing the text itself.
    - First, we will manually clean the text with some relatively simple functions
    - Second, we will use Spacy package that is commonly used in industrisal context (for Language Processing purposes).
    - See intro to Spacy: https://spacy.io/usage/spacy-101

In [53]:
#Download spacy french vocabulary 
!python -m spacy download fr_core_news_md
#They also have multi-lingual stuff--> Can be interesting if we want to apply the method to other types of text

     |████████████████████████████████| 85.7MB 4.9MB/s eta 0:00:01    |                                | 92kB 1.6MB/s eta 0:00:54     |███████▌                        | 20.2MB 3.4MB/s eta 0:00:20     |████████▊                       | 23.4MB 6.3MB/s eta 0:00:10     |██████████▊                     | 28.6MB 2.2MB/s eta 0:00:27     |████████████▌                   | 33.4MB 4.3MB/s eta 0:00:13     |█████████████▋                  | 36.4MB 5.1MB/s eta 0:00:10     |████████████████▍               | 43.8MB 8.2MB/s eta 0:00:06     |██████████████████              | 48.1MB 2.0MB/s eta 0:00:19     |████████████████████████▊       | 66.3MB 290kB/s eta 0:01:07     |█████████████████████████▍      | 68.0MB 2.2MB/s eta 0:00:08     |███████████████████████████▎    | 73.1MB 1.5MB/s eta 0:00:09
  Created wheel for fr-core-news-md: filename=fr_core_news_md-2.1.0-cp37-none-any.whl size=87463874 sha256=22db44441a10f45135e62d4c910853cc5255fe75d92a153b6111d823125b03c4
  Stored in directory: /private/var/fo

In [7]:
try:
    spacy.load('fr_core_news_md')
except Exception as e:
    print(e)
import fr_core_news_md
nlp = fr_core_news_md.load()

In [102]:
#nlp(clean_df.iloc[-2].text.replace(u'\xa0', u'').replace('#', '').replace("  ", " ").replace("  ", " "))

In [8]:
def clean_text(text_):
    """
    Function to remove line break, double spaces...
    """
    text_ = text_.rstrip() 
    text_ = text_.replace(u'\xa0', u'') #Latin normalization
    text_ = text_.replace('\n', ' ')
    text_ = text_.replace('#', '')
    text_ = text_.replace("  ", " ")
    
    text_ = text_.replace('[^\w\s]','')
    #lower_ = lambda x: " ".join(x.lower() for x in x.split())
    #text = lower_(text)
    doc = nlp(text_)
    return doc

Text = clean_df.iloc[-10].text
doc = clean_text(Text)
print('Named Entities found in the last of our articles')
from collections import Counter
count = Counter()
count.update(map(lambda x : x.text, doc.ents))
count

Named Entities found in the last of our articles


Counter({'Madiambal Diagne': 2,
         'Union internationale de la Presse francophone': 1,
         'UPF': 4,
         'Etat togolais': 1,
         'Faure Gnassingbé': 1,
         'Covid-19': 2,
         'M.Diagne': 2,
         'Etats': 1,
         'Organisation internationale de la francophonie': 1,
         'Oif': 1,
         'ONU': 1,
         'UNESCO': 1,
         'ACCT': 1,
         'OIF': 1,
         'Sénégal': 1,
         'Edem Etonam EKUE': 1})

# We will follow up with an analysis of the certain key words with respect to time 

In [9]:
key_words = ['Corona', 'Covid19', 'Covid', 'Covid-19', 'Virus', 'Pandémie', 'Maladie', 'Coronavirus']
low_key_words = map(lambda x: x.lower(), key_words)
key_words.extend(list(low_key_words))
key_words

['Corona',
 'Covid19',
 'Covid',
 'Covid-19',
 'Virus',
 'Pandémie',
 'Maladie',
 'Coronavirus',
 'corona',
 'covid19',
 'covid',
 'covid-19',
 'virus',
 'pandémie',
 'maladie',
 'coronavirus']

In [10]:
import datetime
import seaborn as sns 
sns.set_style("whitegrid")

def clean_date(date_str):
    """
    Modify the date into a timestamp format
    """
    dmy, hms = date_str.split('T')
    dmy = "/".join(dmy.split('-')[::-1])
    element = datetime.datetime.strptime(dmy,
                                         "%d/%m/%Y") 
    return element
    
#clean_df['Date'] = clean_df['Date'].apply(clean_date)


In [11]:
#Sort by Date
clean_df = clean_df.sort_values('Date')
clean_df.reset_index(drop=True, inplace=True)
clean_df

,title,text,Date,country,source,tweet,id
0,Présidentielle au Togo: Le président de la Cén...,"Tchambakou Ayassor, président de la Commission...",2020-02-22T20:42:53,Togo,SAVOIRNEWS1,"Plus de 3,6 millions de Togolais étaient appel...",Togo_SAVOIRNEWS1_22
1,Présidentielle : Le gouvernement dénonce le co...,"Le gouvernement a dénoncé ce samedi, le compor...",2020-02-23T19:53:43,Togo,SAVOIRNEWS1,"Le gouvernement a dénoncé ce samedi, le compor...",Togo_SAVOIRNEWS1_21
2,"Présidentielle au Togo: ""Une avance irrattrapa...",Le président Faure Gnassingbé a enregistré une...,2020-02-23T20:49:31,Togo,SAVOIRNEWS1,http://www.savoirnews.net/presidentielle-au-to...,Togo_SAVOIRNEWS1_20
3,Présidentielle : Pouvoir et opposition revendi...,Le parti au pouvoir et le candidat Agbéyomé Ko...,2020-02-23T20:53:05,Togo,SAVOIRNEWS1,Le parti au pouvoir et le candidat Agbéyomé Ko...,Togo_SAVOIRNEWS1_19
4,Présidentielle au Togo: Faure Gnassingbé réélu...,Faure Essozimna Gnassingbé a été réélu avec 72...,2020-02-24T02:52:34,Togo,SAVOIRNEWS1,Faure Essozimna Gnassingbé a été réélu avec 72...,Togo_SAVOIRNEWS1_18
5,Changements climatiques en Afrique : Réunies à...,Plus de 300 journalistes africaines « Les Pana...,2020-03-07T07:30:42,Togo,SAVOIRNEWS1,Urgence climatique : Les journalistes panafric...,Togo_SAVOIRNEWS1_3
6,LA PREMIERE AGENCE DE PRESSE PRIVEE AU TOGO,Faure Gnassingbé : « La victoire dans la lutte...,2020-03-15T13:51:10,Togo,SAVOIRNEWS1,Le nouveau coronavirus a fait au moins 5.402 m...,Togo_SAVOIRNEWS1_2
7,Coronavirus: Les Évêques du Togo dictent des d...,"Les Évêques du Togo ont annoncé lundi, une sér...",2020-03-17T16:53:44,Togo,SAVOIRNEWS1,Pandémie de coronavirus :\nLes Évêques du #Tog...,Togo_SAVOIRNEWS1_1
8,Le célèbre saxophoniste Manu Dibango est mort ...,"Manu Dibango, saxophoniste camerounais et lége...",2020-03-24T10:45:16,Togo,SAVOIRNEWS1,"Manu Dibango, saxophoniste camerounais et lége...",Togo_SAVOIRNEWS1_17
9,"Coronavirus au Togo: 20 cas, le SYNPHOT offre ...",Le Syndicat national des praticiens hospitalie...,2020-03-24T13:16:54,Togo,SAVOIRNEWS1,Contre le #Covid19 Le Syndicat national des pr...,Togo_SAVOIRNEWS1_16


In [12]:
%matplotlib notebook
from ipywidgets import interact
import plotly.graph_objects as go

@interact(index=(0,len(clean_df)))
def visualize(index):
    """
    Functioo to get histograms 
    the key words we're interested in
    and the main topics more generally speaking.
    """
    df = clean_df
    count = Counter()
    Text = df.iloc[index].text
    Title = df.iloc[index].title
    doc = clean_text(Title+Text)
    key_words = ['Corona', 'Covid19', 
                 'Covid', 'Covid-19', 
                 'Virus', 'Pandémie', 
                 'Maladie', 'Coronavirus', 
                'Santé']
    low_key_words = map(lambda x: x.lower(), key_words)
    key_words.extend(list(low_key_words))
    #print(Text)
    print(Title, df.iloc[index].Date)
    count = Counter()
    count_key = Counter()
    for token in doc:
        if token.pos_ in ['PROPN', 'NOUN', 'VERB', 'ADJ']:
            #print(token.text, token.pos_, token.dep_)
            count.update([token.text.lower()])
        if token.text in key_words:
            count_key.update([token.text.lower()])
    print(count_key)

    labels, values = zip(*count.most_common()[:10])
    fig = go.Figure(data=[go.Histogram(x=labels, y=values, histfunc='sum')])
    fig.show()
    try:
        labels_key, values_key = zip(*count_key.most_common()[:10])
        fig = go.Figure(data=[go.Histogram(x=labels_key, y=values_key, histfunc='sum')])
        fig.show()
    except:
        print('No "key word" in this doc')
  
#@interact(index=(0,70)    
#visualize(40)

interactive(children=(IntSlider(value=11, description='index', max=23), Output()), _dom_classes=('widget-inter…

In [13]:
def get_stats(index):
    """
    Function to count the key words found in the text with time. 
    Add a new column to the dataframe.
    """
    df = clean_df
    count = Counter()
    Text = df.iloc[index].text
    Title = df.iloc[index].title
    date = df.iloc[index].Date
    doc = clean_text(Title+Text)
    key_words = ['Corona', 'Covid19', 
                 'Covid', 'Covid-19', 
                 'Virus', 'Pandémie', 
                 'Maladie', 'Coronavirus', 
                'Santé']
    low_key_words = map(lambda x: x.lower(), key_words)
    key_words.extend(list(low_key_words))
    print(Title, df.iloc[index].Date)
    count = Counter()
    count_key = Counter()
    for token in doc:
        if token.pos_ in ['PROPN', 'NOUN', 'VERB', 'ADJ']:
            #print(token.text, token.pos_, token.dep_)
            count.update([token.text.lower()])
        if token.text in key_words:
            count_key.update([token.text.lower()])
    return Title, count_key, date

freq_ = []
for index in range(len(clean_df)):
    _, counter, date = get_stats(index)
    freq_.append(sum(counter.values()))

Présidentielle au Togo: Le président de la Céni appelle au respect "strict" des procédures de vote 2020-02-22T20:42:53
Présidentielle : Le gouvernement dénonce le comportement d’une organisation de "pervertir" le processus 2020-02-23T19:53:43
Présidentielle au Togo: "Une avance irrattrapable pour Faure Gnassingbé" (Gilbert Bawara) 2020-02-23T20:49:31
Présidentielle : Pouvoir et opposition revendiquent la victoire, la Céni annonce les "résultats provisoires" lundi au plus tard 2020-02-23T20:53:05
Présidentielle au Togo: Faure Gnassingbé réélu avec 72,36% des voix (Officiel) 2020-02-24T02:52:34
Changements climatiques en Afrique : Réunies à Casablanca, "Les Panafricaines" aptes à se mobiliser toute l’année 2020 2020-03-07T07:30:42
LA PREMIERE AGENCE DE PRESSE PRIVEE AU TOGO 2020-03-15T13:51:10
Coronavirus: Les Évêques du Togo dictent des dispositions pour les messes 2020-03-17T16:53:44
Le célèbre saxophoniste Manu Dibango est mort des suites du coronavirus 2020-03-24T10:45:16
Coronavirus

In [14]:
clean_df['freq'] = freq_
import plotly.express as px
#tdf = px.data.tips()
fig = px.histogram(clean_df, 
                   x="Date", 
                   y="freq", 
                   marginal="rug", 
                   histfunc='sum', 
                   color='freq',  
                   nbins=20)#box, rug
fig.show()

In [15]:
"""Source of info for plots: https://plotly.com/python/plotly-express/"""

'Source of info for plots: https://plotly.com/python/plotly-express/'

In [16]:
fig = px.scatter(clean_df, x="Date", 
                 y="freq", 
                 size="freq", 
                 color="freq",
                 log_x=False, size_max=60)
fig.show()

In [137]:
import plotly.express as px
#df = px.data.gapminder()
fig = px.scatter(clean_df, x="freq", y="freq", animation_frame="Date",
           size="freq", 
           log_x=False, 
            size_max=45, range_x=[0,100], range_y=[0,20])
fig.show()

In [ ]:
!pwd

# Replicate the above with one single file


In [149]:
from lib import prepro
import imp
imp.reload(prepro)

<module 'lib.prepro' from '../lib/prepro.py'>

In [150]:
#example_article = 'articles_JourDuMali_2020-02-01_2020-04-14.json'
#example_path = os.path.join('../data', example_article)
#clean_df = prepro.main(example_path)

In [17]:
#clean_df = prepro.main(read_path=example_path)

# Simple Pipeline with Camembert?
# Need to fix that part

In [20]:
import transformers
#from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer,AutoModelForQuestionAnswering

# Sentiment analysis pipeline
#pipeline('sentiment-analysis')

# Question answering pipeline, specifying the checkpoint identifier
#pipeline('question-answering', model='distilbert-base-cased-distilled-squad', tokenizer='bert-base-cased')

# Named entity recognition pipeline, passing in a specific model and tokenizer
#model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME)
#tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
#pip=pipeline('ner', model=model, tokenizer=tokenizer)

In [22]:
help(transformers)

AttributeError: module 'transformers' has no attribute 'pipeline'

In [129]:
#from transformers import pipeline
from transformers import (AutoModelForQuestionAnswering, 
                          AutoModelForSequenceClassification, 
                          AutoTokenizer, 
                          AutoModelWithLMHead)


In [49]:
tokenizer = AutoTokenizer.from_pretrained("camembert-base")
model = AutoModelForSequenceClassification.from_pretrained("camembert-base")

In [33]:
from transformers import SummarizationPipeline

ImportError: cannot import name 'SummarizationPipeline' from 'transformers' (/Users/theoguenais/anaconda/envs/hurricast/lib/python3.7/site-packages/transformers/__init__.py)

In [82]:
t1 = t1.replace('\n', ' ').replace('  ',' ')
t2 = t2.replace('\n', ' ').replace('  ',' ')


In [181]:
t1 = clean_df.iloc[-6].text
t2 = clean_df.iloc[-4].text

In [171]:
t = tokenizer.encode_plus(t1,t2, return_tensors="pt",max_length=512)
t.keys()
t['input_ids'].size()
#del t['']
#t.keys()
#del t['special_tokens_mask']

torch.Size([1, 512])

In [176]:
torch.softmax(model(t['input_ids'])[0], axis=1)*100

tensor([[48.0970, 51.9030]], grad_fn=<MulBackward0>)

In [186]:
#from transformers import pipeline
!conda list 

# packages in environment at /Users/theoguenais/anaconda/envs/hurricast:
#
# Name                    Version                   Build  Channel
_pytorch_select           0.1                       cpu_0  
appdirs                   1.4.3                    pypi_0    pypi
appnope                   0.1.0                    pypi_0    pypi
arviz                     0.5.1                    pypi_0    pypi
asn1crypto                0.24.0                py37_1003    conda-forge
astroid                   2.2.5                    py37_0  
atomicwrites              1.3.0                    pypi_0    pypi
attrs                     19.1.0                     py_0    conda-forge
autodiff-addictedtocs     0.1                      pypi_0    pypi
autograd                  1.3                      pypi_0    pypi
autopep8                  1.4.4                      py_0  
backcall                  0.1.0                    pypi_0    pypi
beautifulsoup4            4.8.0                    py37_0    anaconda


In [188]:
!ls ../data

Macky_Sall_twitter.json
articles_Gabonews_com_2020-02-01_2020-04-14.json
articles_ICIBrazza_2020-02-01_2020-04-14.json
articles_JourDuMali_2020-02-01_2020-04-14.json
articles_LeSoleilonline_2020-01-01_2020-04-14.json
articles_Lefaso_net_2020-02-01_2020-04-14.json
articles_SAVOIRNEWS1_2020-02-01_2020-04-14.json
articles_comoresinfos_2020-02-01_2020-04-14.json
articles_iwacuinfo_2020-02-01_2020-04-14.json
articles_madatribune_2020-02-01_2020-04-14.json
lequotidien.json
lesoleil.json
tweets_CongoActu_2020-02-01_2020-04-14.json
tweets_Gabonews_com_2020-02-01_2020-04-14.json
tweets_InfosGabon_2020-02-01_2020-04-14.json
tweets_JourDuMali_2020-02-01_2020-04-14.json
tweets_LeSoleilonline_2020-01-01_2020-04-14.json
tweets_LeSoleilonline_2020-02-01_2020-04-12.json
tweets_LeSoleilonline_2020-02-02_2020-04-12.json
tweets_Lefaso_net_2020-02-01_2020-04-14.json
tweets_SAVOIRNEWS1_2020-02-01_2020-04-14.json
tweets_madatribune_2020-02-01_2020-04-14.json


In [190]:
model = AutoModelWithLMHead.from_pretrained("camembert-base")
tokenizer = AutoTokenizer.from_pretrained("camembert-base")
ARTICLE = clean_df.iloc[-10].text
# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer.encode("summarize: " + ARTICLE, return_tensors="pt", max_length=512)
outputs = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
print(outputs)

AttributeError: 'CamembertForMaskedLM' object has no attribute 'generate'

In [191]:
questions = ['Combien de décès ont été enregistrés au Togo ?',
             'Comment décrire la réaction du gouvernement ?',
             'Qelles types de rassemblements devraient être annulés']


'Le Togo a déjà enregistré 30 cas dont un décès et sept personnes guéries, indique le dernier bilan officiel publié ce dimanche.\n\nSelon Komi Sélom Klassou (Premier ministre), ces nouvelles mesures permettront de « mieux protéger et à temps opportun nos concitoyens et atténuer les conséquences de cette crise sanitaire sur la vie économique, sociale, culturelle et sécuritaire du pays « .\n\n« La riposte engagée contre la maladie de coronavirus Covid -19 commande une extrême réactivité et une capacité exceptionnelle d’adaptation des différentes mesures de prévention et de protection prises, ainsi qu’une mobilisation de moyens de tous ordres en vue de trouver les solutions qui permettent d’endiguer durablement les effets de l’épidémie », a-t-il souligné, sans dévoiler le contenu de ces nouvelles mesures.\n\n‘Aux grands maux, les grands remèdes’\n\nSelon le chef du gouvernement, le régime d’habilitation sollicité n’a qu’un seul objectif, « celui de mieux protéger et à temps opportun nos c

In [200]:
tokenizer = AutoTokenizer.from_pretrained("camembert-base")
model = FlaubertForQuestionAnswering.from_pretrained("flaubert-base")

NameError: name 'FlaubertForQuestionAnswering' is not defined

In [201]:
config = FlaubertConfig.from_pretrained(MODEL_NAME)

NameError: name 'FlaubertConfig' is not defined

In [202]:
MODEL_NAME = "camembert-base"
# 2 POSSIBILITIES TO LOAD CAMEMBERT

#1st
config = CamembertConfig.from_pretrained(MODEL_NAME)    # Download configuration from S3 and cache.
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_config(config) 

NameError: name 'CamembertConfig' is not defined

In [205]:
from transformers import CamembertConfig

In [208]:
!pip install transformers --upgrade

  Using cached transformers-2.8.0-py3-none-any.whl (563 kB)
  Using cached tokenizers-0.5.2.tar.gz (64 kB)
  Installing build dependencies ... error
  ERROR: Command errored out with exit status 1:
   command: /Users/theoguenais/anaconda/envs/hurricast/bin/python /Users/theoguenais/anaconda/envs/hurricast/lib/python3.7/site-packages/pip install --ignore-installed --no-user --prefix /private/var/folders/gt/25j8ndxj1c52vv_v5bv0b_n80000gn/T/pip-build-env-pxlh8ho9/overlay --no-warn-script-location --no-binary :none: --only-binary :none: -i https://pypi.org/simple -- setuptools wheel setuptools-rust
       cwd: None
  Complete output (14 lines):
  Traceback (most recent call last):
    File "/Users/theoguenais/anaconda/envs/hurricast/lib/python3.7/runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "/Users/theoguenais/anaconda/envs/hurricast/lib/python3.7/runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "/Users/theoguenais/anaconda/envs/hu

In [ ]:
#!pip uninstall enum34
#!pip install --upgrade pip

# Gathering data accross countries